In [1]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# 加載數據
file_path = '104Data/IVB_285.csv'
df = pd.read_csv(file_path)

# 定義計算函數
def get_R_fit(df_group):
    df_group = df_group.reset_index(drop=True)
    df_group['dV/dI'] = pd.DataFrame(np.diff(df_group['meas_voltage_K2']) / np.diff(df_group['appl_current']), columns=['r']).reindex(df_group.index, method=None)
    peaks, _ = find_peaks(df_group['dV/dI'])
    # 提取峰值電流和峰值微分電壓值
    peak_current = [df_group['appl_current'][i] for i in peaks]
    peak_R = [df_group['dV/dI'][i] for i in peaks]

    # 根據值對峰進行排序
    sorted_peak_indices = np.argsort(peak_R)[::-1]  # 降序排序
    top_two_peak_indices = sorted_peak_indices[:2]  # 選擇前兩個索引

    # 提取前兩個峰
    top_two_peak_current = [peak_current[i] for i in top_two_peak_indices]
    top_two_peak_R = [peak_R[i] for i in top_two_peak_indices]
    
    # 確保 peak0 小於 peak1
    peak0, peak1 = top_two_peak_current
    if peak1 < peak0:
        peak0, peak1 = peak1, peak0
        top_two_peak_current = [peak0, peak1]
    
    df_0 = df_group[df_group['appl_current'] < peak0]
    df_1 = df_group[df_group['appl_current'] > peak1]

    fit_0 = np.polyfit(df_0['appl_current'], df_0['meas_voltage_K2'], 1)
    fit_1 = np.polyfit(df_1['appl_current'], df_1['meas_voltage_K2'], 1)
    R_fit0 = fit_0[0]
    R_fit1 = fit_1[0]
    R_fit = (R_fit0 + R_fit1) / 2
    
    current_0_index = df_group['appl_current'].abs().idxmin()
    Ic = top_two_peak_current[1] - df_group['appl_current'].iloc[current_0_index]
    IcRn = Ic * R_fit

    return pd.Series({
        'R_fit': R_fit,
        'R_fit0': R_fit0,
        'R_fit1': R_fit1,
        'top_two_peak_current': top_two_peak_current,
        'Ic': Ic,
        'IcRn': IcRn,
        'current_0_index': current_0_index
    })

# 對 y_field 進行分組並應用計算函數
results = df.groupby('y_field').apply(get_R_fit).reset_index()

# 將結果與原始數據合併
df_merged = df.merge(results, on='y_field', how='left')
df_merged.head()


/var/folders/_b/y4_l8xx96yd_n1482vw0pkpc0000gn/T/ipykernel_81810/1826716468.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = df.groupby('y_field').apply(get_R_fit).reset_index()


,Unnamed: 0,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,0,-0.002,-0.000030,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
1,1,-0.002,-0.000030,-0.000032,1.28090,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
2,2,-0.002,-0.000030,-0.000031,1.27020,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
3,3,-0.002,-0.000029,-0.000031,1.06770,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
4,4,-0.002,-0.000029,-0.000031,1.29065,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150


In [2]:
import numpy as np
from scipy.signal import find_peaks

# Define the calculation function
def get_R_fit(df_group):
    df_group = df_group.reset_index(drop=True)
    df_group['dV/dI'] = pd.DataFrame(np.diff(df_group['meas_voltage_K2']) / np.diff(df_group['appl_current']), columns=['r']).reindex(df_group.index, method=None)
    peaks, _ = find_peaks(df_group['dV/dI'])
    if len(peaks) < 2:
        return pd.Series({
            'R_fit': np.nan,
            'R_fit0': np.nan,
            'R_fit1': np.nan,
            'top_two_peak_current': [np.nan, np.nan],
            'Ic': np.nan,
            'IcRn': np.nan,
            'current_0_index': np.nan
        })
    # Extract peak current and peak differential voltage values
    peak_current = [df_group['appl_current'][i] for i in peaks]
    peak_R = [df_group['dV/dI'][i] for i in peaks]

    # Sort the peaks based on their values
    sorted_peak_indices = np.argsort(peak_R)[::-1]  # Sort in descending order
    top_two_peak_indices = sorted_peak_indices[:2]  # Select the top two indices

    # Extract the top two peaks
    top_two_peak_current = [peak_current[i] for i in top_two_peak_indices]
    top_two_peak_R = [peak_R[i] for i in top_two_peak_indices]
    
    # Ensure peak0 is less than peak1
    peak0, peak1 = top_two_peak_current
    if peak1 < peak0:
        peak0, peak1 = peak1, peak0
        top_two_peak_current = [peak0, peak1]
    
    df_0 = df_group[df_group['appl_current'] < peak0]
    df_1 = df_group[df_group['appl_current'] > peak1]

    fit_0 = np.polyfit(df_0['appl_current'], df_0['meas_voltage_K2'], 1)
    fit_1 = np.polyfit(df_1['appl_current'], df_1['meas_voltage_K2'], 1)
    R_fit0 = fit_0[0]
    R_fit1 = fit_1[0]
    R_fit = (R_fit0 + R_fit1) / 2
    
    current_0_index = df_group['appl_current'].abs().idxmin()
    Ic = top_two_peak_current[1] - df_group['appl_current'].iloc[current_0_index]
    IcRn = Ic * R_fit

    return pd.Series({
        'R_fit': R_fit,
        'R_fit0': R_fit0,
        'R_fit1': R_fit1,
        'top_two_peak_current': top_two_peak_current,
        'Ic': Ic,
        'IcRn': IcRn,
        'current_0_index': current_0_index
    })

# Group by y_field and apply the calculation function
results = df.groupby('y_field').apply(get_R_fit).reset_index()

# Merge the results with the original dataframe
df_merged = df.merge(results, on='y_field', how='left')
df_merged.head()


/var/folders/_b/y4_l8xx96yd_n1482vw0pkpc0000gn/T/ipykernel_81810/132670409.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = df.groupby('y_field').apply(get_R_fit).reset_index()


,Unnamed: 0,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,0,-0.002,-0.000030,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
1,1,-0.002,-0.000030,-0.000032,1.28090,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
2,2,-0.002,-0.000030,-0.000031,1.27020,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
3,3,-0.002,-0.000029,-0.000031,1.06770,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
4,4,-0.002,-0.000029,-0.000031,1.29065,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150


In [3]:
df_merged

,Unnamed: 0,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,0,-0.002,-0.000030,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
1,1,-0.002,-0.000030,-0.000032,1.28090,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
2,2,-0.002,-0.000030,-0.000031,1.27020,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
3,3,-0.002,-0.000029,-0.000031,1.06770,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
4,4,-0.002,-0.000029,-0.000031,1.29065,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
...,...,...,...,...,...,...,...,...,...,...,...,...
60496,60496,-0.004,0.000029,0.000032,1.25185,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60497,60497,-0.004,0.000029,0.000032,0.94015,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60498,60498,-0.004,0.000030,0.000032,1.56765,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60499,60499,-0.004,0.000030,0.000033,1.24620,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150


In [18]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# 加載數據
file_path = '104Data/IVB_285.csv'
df = pd.read_csv(file_path)

# 定義計算函數
def get_R_fit(df_group):
    df_group = df_group.reset_index(drop=True)
    df_group['dV/dI'] = pd.DataFrame(np.diff(df_group['meas_voltage_K2']) / np.diff(df_group['appl_current']), columns=['r']).reindex(df_group.index, method=None)
    peaks, _ = find_peaks(df_group['dV/dI'])
    if len(peaks) < 2:
        return pd.Series({
            'R_fit': np.nan,
            'R_fit0': np.nan,
            'R_fit1': np.nan,
            'top_two_peak_current': [np.nan, np.nan],
            'Ic': np.nan,
            'IcRn': np.nan,
            'current_0_index': np.nan
        })
    # 提取峰值電流和峰值微分電壓值
    peak_current = [df_group['appl_current'][i] for i in peaks]
    peak_R = [df_group['dV/dI'][i] for i in peaks]

    # 根據值對峰進行排序
    sorted_peak_indices = np.argsort(peak_R)[::-1]  # 降序排序
    top_two_peak_indices = sorted_peak_indices[:2]  # 選擇前兩個索引

    # 提取前兩個峰
    top_two_peak_current = [peak_current[i] for i in top_two_peak_indices]
    top_two_peak_R = [peak_R[i] for i in top_two_peak_indices]
    
    # 確保 peak0 小於 peak1
    peak0, peak1 = top_two_peak_current
    if peak1 < peak0:
        peak0, peak1 = peak1, peak0
        top_two_peak_current = [peak0, peak1]
    
    df_0 = df_group[df_group['appl_current'] < peak0]
    df_1 = df_group[df_group['appl_current'] > peak1]

    fit_0 = np.polyfit(df_0['appl_current'], df_0['meas_voltage_K2'], 1)
    fit_1 = np.polyfit(df_1['appl_current'], df_1['meas_voltage_K2'], 1)
    R_fit0 = fit_0[0]
    R_fit1 = fit_1[0]
    R_fit = (R_fit0 + R_fit1) / 2
    
    current_0_index = df_group['appl_current'].abs().idxmin()
    Ic = top_two_peak_current[1] - df_group['appl_current'].iloc[current_0_index]
    IcRn = Ic * R_fit

    return pd.Series({
        'R_fit': R_fit,
        'R_fit0': R_fit0,
        'R_fit1': R_fit1,
        'top_two_peak_current': top_two_peak_current,
        'Ic': Ic,
        'IcRn': IcRn,
        'current_0_index': current_0_index
    })

# 對 y_field 進行分組並應用計算函數
results = df.groupby('y_field').apply(get_R_fit).reset_index()

# 將結果與原始數據合併
df_merged = df.merge(results, on='y_field', how='left').drop(columns=['Unnamed: 0'])

# 只保留每個 y_field 對應的第一行計算結果
df_unique = df_merged.drop_duplicates(subset=['y_field'])

# 保存結果
df_unique.to_csv('104Data/IVB_285_processed.csv', index=False)
df_unique.head()


/var/folders/_b/y4_l8xx96yd_n1482vw0pkpc0000gn/T/ipykernel_81810/205026568.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = df.groupby('y_field').apply(get_R_fit).reset_index()


,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,-0.00200,-0.00003,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
301,-0.00201,-0.00003,-0.000032,2.40245,1.178163,1.132380,1.223946,"[-4e-06, 1.46e-05]",0.000015,0.000017,150
602,-0.00202,-0.00003,-0.000032,2.51940,1.173209,1.133674,1.212744,"[-4.2e-06, 1.48e-05]",0.000015,0.000017,150
903,-0.00203,-0.00003,-0.000032,2.43905,1.174816,1.133202,1.216431,"[-4.2e-06, 1.5e-05]",0.000015,0.000018,150
1204,-0.00204,-0.00003,-0.000032,2.35640,1.176564,1.134052,1.219076,"[-4.2e-06, 1.54e-05]",0.000015,0.000018,150


In [ ]:
# Remove the 'Unnamed: 0' column
unique_results_cleaned = unique_results.drop(columns=['Unnamed: 0'])

# Save the processed data without the 'Unnamed: 0' column
processed_file_path_cleaned = '/mnt/data/IVB_285_processed_cleaned.csv'
unique_results_cleaned.to_csv(processed_file_path_cleaned, index=False)

# Display the first few rows of the cleaned results
unique_results_cleaned.head()

In [16]:
# Group by y_field and apply the calculation function
results = df.groupby('y_field').apply(get_R_fit).reset_index()

# Merge the results with the original dataframe
df_merged = df.merge(results, on='y_field', how='left')

# Verify unique values of y_field and their corresponding results
unique_results = df_merged.drop_duplicates(subset=['y_field']).drop(columns=['Unnamed: 0'])

# Save the processed data
processed_file_path = '104Data/IVB_285_processed.csv'
unique_results.to_csv(processed_file_path, index=False)

# Display the first few rows of the unique results
unique_results.head()


/var/folders/_b/y4_l8xx96yd_n1482vw0pkpc0000gn/T/ipykernel_81810/2606246561.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = df.groupby('y_field').apply(get_R_fit).reset_index()


,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,-0.00200,-0.00003,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
301,-0.00201,-0.00003,-0.000032,2.40245,1.178163,1.132380,1.223946,"[-4e-06, 1.46e-05]",0.000015,0.000017,150
602,-0.00202,-0.00003,-0.000032,2.51940,1.173209,1.133674,1.212744,"[-4.2e-06, 1.48e-05]",0.000015,0.000017,150
903,-0.00203,-0.00003,-0.000032,2.43905,1.174816,1.133202,1.216431,"[-4.2e-06, 1.5e-05]",0.000015,0.000018,150
1204,-0.00204,-0.00003,-0.000032,2.35640,1.176564,1.134052,1.219076,"[-4.2e-06, 1.54e-05]",0.000015,0.000018,150


In [8]:
unique_results

,Unnamed: 0,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,0,-0.00200,-0.00003,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
301,301,-0.00201,-0.00003,-0.000032,2.40245,1.178163,1.132380,1.223946,"[-4e-06, 1.46e-05]",0.000015,0.000017,150
602,602,-0.00202,-0.00003,-0.000032,2.51940,1.173209,1.133674,1.212744,"[-4.2e-06, 1.48e-05]",0.000015,0.000017,150
903,903,-0.00203,-0.00003,-0.000032,2.43905,1.174816,1.133202,1.216431,"[-4.2e-06, 1.5e-05]",0.000015,0.000018,150
1204,1204,-0.00204,-0.00003,-0.000032,2.35640,1.176564,1.134052,1.219076,"[-4.2e-06, 1.54e-05]",0.000015,0.000018,150
...,...,...,...,...,...,...,...,...,...,...,...,...
58996,58996,-0.00396,-0.00003,-0.000032,2.46655,1.152408,1.127212,1.177604,"[-3e-06, 9.6e-06]",0.000010,0.000011,150
59297,59297,-0.00397,-0.00003,-0.000032,2.57805,1.153727,1.127295,1.180159,"[-3e-06, 9.2e-06]",0.000009,0.000011,150
59598,59598,-0.00398,-0.00003,-0.000032,2.47955,1.153824,1.127179,1.180470,"[-3e-06, 8.8e-06]",0.000009,0.000010,150
59899,59899,-0.00399,-0.00003,-0.000032,2.39765,1.153016,1.125533,1.180499,"[-2.8e-06, 8.4e-06]",0.000008,0.000010,150


In [15]:
df_merged.reset_index()

,index,Unnamed: 0,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,0,0,-0.002,-0.000030,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
1,1,1,-0.002,-0.000030,-0.000032,1.28090,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
2,2,2,-0.002,-0.000030,-0.000031,1.27020,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
3,3,3,-0.002,-0.000029,-0.000031,1.06770,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
4,4,4,-0.002,-0.000029,-0.000031,1.29065,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60496,60496,60496,-0.004,0.000029,0.000032,1.25185,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60497,60497,60497,-0.004,0.000029,0.000032,0.94015,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60498,60498,60498,-0.004,0.000030,0.000032,1.56765,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60499,60499,60499,-0.004,0.000030,0.000033,1.24620,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150


In [12]:
df_merged

,Unnamed: 0,y_field,appl_current,meas_voltage_K2,dV/dI,R_fit,R_fit0,R_fit1,top_two_peak_current,Ic,IcRn,current_0_index
0,0,-0.002,-0.000030,-0.000032,2.46220,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
1,1,-0.002,-0.000030,-0.000032,1.28090,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
2,2,-0.002,-0.000030,-0.000031,1.27020,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
3,3,-0.002,-0.000029,-0.000031,1.06770,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
4,4,-0.002,-0.000029,-0.000031,1.29065,1.169855,1.139147,1.200564,"[-4e-06, 1.44e-05]",0.000014,0.000017,150
...,...,...,...,...,...,...,...,...,...,...,...,...
60496,60496,-0.004,0.000029,0.000032,1.25185,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60497,60497,-0.004,0.000029,0.000032,0.94015,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60498,60498,-0.004,0.000030,0.000032,1.56765,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
60499,60499,-0.004,0.000030,0.000033,1.24620,1.149087,1.126353,1.171821,"[-2.8e-06, 8.2e-06]",0.000008,0.000009,150
